In [7]:
import os

import pandas as pd

import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom 

In [8]:
def create_osm(df, osm_file):
    osm = ET.Element("osm")
    osm.set("version", "0.6")
    osm.set("generator", "CGImap 0.9.2 (596732 spike-08.openstreetmap.org)")
    osm.set("copyright", "OpenStreetMap and contributors")
    osm.set("attribution", "http://www.openstreetmap.org/copyright")
    osm.set("license", "http://opendatacommons.org/licenses/odbl/1-0/")

    nodes_df = df\
    .drop_duplicates(subset=['node_id'])\
    .sort_values(by='node_id')

    for _, row in nodes_df.iterrows():
        node_id = row["node_id"]

        ET.SubElement(
            osm,
            "node",
            id=str(node_id),
            lat=str(row["lat"]),
            lon=str(row["lon"]),
            visible="true",
            version="1",        
        )

    tags = [
        {"k": "highway", "v": "footway"},
    ]

    ways_df = df.groupby('filename')['node_id'].agg(lambda x: list(x.unique())).reset_index()
    
    for idx, row in ways_df.iterrows():
        way_id = idx + 1

        way = ET.SubElement(
            osm,
            "way",
            id=str(way_id),
            visible="true",
            version="1",
        )

        for node_id in row["node_id"]:
            nd = ET.SubElement(way, "nd")
            nd.set("ref", str(node_id))

        for tag_data in tags:
            tag = ET.SubElement(way, "tag")
            tag.set("k", tag_data["k"])
            tag.set("v", tag_data["v"]) 

        xml_str = minidom.parseString(ET.tostring(osm)).toprettyxml(indent="  ")

    with open(osm_file, "w") as f:
          f.write(xml_str)

In [9]:
output_dir = '../osm'

node_dfs = []

for csv_file in [
    '../nodes/gpx-nodes.csv',
    '../nodes/common-osm-nodes.csv',
]:
    df = pd.read_csv(csv_file)
    node_dfs.append(df)

node_dfs = pd.concat(node_dfs).reset_index(drop=True)  

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

create_osm(
  node_dfs,
  output_dir + '/custom.osm',
)